# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)


pd.set_option('display.max_columns', None)

df.head()



loan_amnt  int_rate  installment home_ownership  annual_inc  \
0    10500.0    0.1719       375.35           RENT     66000.0   
1    25000.0    0.2000       929.09       MORTGAGE    105000.0   
2    20000.0    0.2000       529.88       MORTGAGE     56000.0   
3    10000.0    0.1640       353.55           RENT     92000.0   
4    22000.0    0.1474       520.39       MORTGAGE     52000.0   

  verification_status   issue_d loan_status pymnt_plan    dti  delinq_2yrs  \
0     Source Verified  Mar-2019    low_risk          n  27.24          0.0   
1            Verified  Mar-2019    low_risk          n  20.23          0.0   
2            Verified  Mar-2019    low_risk          n  24.26          0.0   
3            Verified  Mar-2019    low_risk          n  31.44          0.0   
4        Not Verified  Mar-2019    low_risk          n  18.76          0.0   

   inq_last_6mths  open_acc  pub_rec  revol_bal  total_acc  \
0             0.0       8.0      0.0     1609.0       14.0   
1             0.0      17.0      1.0    18368.0       34.0   
2             0.0       8.0      0.0    13247.0       21.0   
3             1.0      10.0      1.0    17996.0       23.0   
4             1.0      14.0      0.0     9091.0       28.0   

  initial_list_status  out_prncp  out_prncp_inv  total_pymnt  total_pymnt_inv  \
0                   w   10204.87       10204.87       355.29           355.29   
1                   w   24293.13       24293.13       873.53           873.53   
2                   w   19636.79       19636.79       485.44           485.44   
3                   w    9719.34        9719.34       335.33           335.33   
4                   w   21614.73       21614.73       484.36           484.36   

   total_rec_prncp  total_rec_int  total_rec_late_fee  recoveries  \
0           295.13          60.16                 0.0         0.0   
1           706.87         166.66                 0.0         0.0   
2           363.21         122.23                 0.0         0.0   
3           280.66          54.67                 0.0         0.0   
4           385.27          99.09                 0.0         0.0   

   collection_recovery_fee  last_pymnt_amnt next_pymnt_d  \
0                      0.0           375.35     May-2019   
1                      0.0           929.09     May-2019   
2                      0.0           529.88     May-2019   
3                      0.0           353.55     May-2019   
4                      0.0           520.39     May-2019   

   collections_12_mths_ex_med  policy_code application_type  acc_now_delinq  \
0                         0.0          1.0       Individual             0.0   
1                         0.0          1.0       Individual             0.0   
2                         0.0          1.0       Individual             0.0   
3                         0.0          1.0       Individual             0.0   
4                         0.0          1.0       Individual             0.0   

   tot_coll_amt  tot_cur_bal  open_acc_6m  open_act_il  open_il_12m  \
0         178.0      38199.0          3.0          4.0          2.0   
1         165.0     228119.0          3.0          3.0          2.0   
2           0.0      45684.0          2.0          3.0          2.0   
3           0.0      68784.0          0.0          4.0          1.0   
4           0.0     177745.0          0.0          1.0          0.0   

   open_il_24m  mths_since_rcnt_il  total_bal_il  il_util  open_rv_12m  \
0          4.0                 3.0       36590.0     59.0          1.0   
1          3.0                 4.0       42273.0     86.0          4.0   
2          3.0                 4.0       32437.0     75.0          2.0   
3          3.0                 9.0       50788.0     66.0          0.0   
4          2.0                14.0       16828.0     84.0          1.0   

   open_rv_24m  max_bal_bc  all_util  total_rev_hi_lim  inq_fi  total_cu_tl  \
0          2.0      1553.0      58.0            3700.0     1.0          0.0

In [5]:
# Create the LabelEncoder instance
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [6]:
# Fitting and encoding the columns with the LabelEncoder

le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

le.fit(df["loan_status"])
df["loan_status"] = le.transform(df["loan_status"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

le.fit(df["issue_d"])
df["issue_d"] = le.transform(df["issue_d"])

le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

le.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = le.transform(df["next_pymnt_d"])

In [7]:
df.issue_d.unique()

array([2, 0, 1], dtype=int64)

In [8]:
df.loc[0]

loan_amnt                     10500.0000
int_rate                          0.1719
installment                     375.3500
home_ownership                    3.0000
annual_inc                    66000.0000
                                 ...    
total_bal_ex_mort             38199.0000
total_bc_limit                 2000.0000
total_il_high_credit_limit    61987.0000
hardship_flag                     0.0000
debt_settlement_flag              0.0000
Name: 0, Length: 86, dtype: float64

# Split the Data into Training and Testing

In [9]:
x_cols = [i for i in df.columns if i not in ('loan_status')]

# Create our features
X = df[x_cols]

# Create our target
y = df['loan_status']

In [10]:
X

loan_amnt  int_rate  installment  home_ownership  annual_inc  \
0        10500.0    0.1719       375.35               3     66000.0   
1        25000.0    0.2000       929.09               1    105000.0   
2        20000.0    0.2000       529.88               1     56000.0   
3        10000.0    0.1640       353.55               3     92000.0   
4        22000.0    0.1474       520.39               1     52000.0   
...          ...       ...          ...             ...         ...   
68812    10000.0    0.1502       346.76               3     26000.0   
68813    12000.0    0.2727       368.37               3     63000.0   
68814     5000.0    0.1992       185.62               1     52000.0   
68815    40000.0    0.0646      1225.24               1    520000.0   
68816    16000.0    0.1131       350.36               1     72000.0   

       verification_status  issue_d  pymnt_plan    dti  delinq_2yrs  \
0                        1        2           0  27.24          0.0   
1                        2        2           0  20.23          0.0   
2                        2        2           0  24.26          0.0   
3                        2        2           0  31.44          0.0   
4                        0        2           0  18.76          0.0   
...                    ...      ...         ...    ...          ...   
68812                    1        1           0   9.60          0.0   
68813                    0        1           0  29.07          0.0   
68814                    1        1           0  14.86          0.0   
68815                    2        1           0   9.96          0.0   
68816                    2        1           0   7.02          2.0   

       inq_last_6mths  open_acc  pub_rec  revol_bal  total_acc  \
0                 0.0       8.0      0.0     1609.0       14.0   
1                 0.0      17.0      1.0    18368.0       34.0   
2                 0.0       8.0      0.0    13247.0       21.0   
3                 1.0      10.0      1.0    17996.0       23.0   
4                 1.0      14.0      0.0     9091.0       28.0   
...               ...       ...      ...        ...        ...   
68812             0.0       9.0      0.0     2684.0       20.0   
68813             0.0       8.0      0.0    13314.0       26.0   
68814             0.0       5.0      1.0     3715.0       14.0   
68815             1.0      21.0      0.0    59529.0       57.0   
68816             0.0      12.0      1.0    11882.0       39.0   

       initial_list_status  out_prncp  out_prncp_inv  total_pymnt  \
0                        1   10204.87       10204.87       355.29   
1                        1   24293.13       24293.13       873.53   
2                        1   19636.79       19636.79       485.44   
3                        1    9719.34        9719.34       335.33   
4                        1   21614.73       21614.73       484.36   
...                    ...        ...            ...          ...   
68812                    1    9326.86        9326.86      1019.42   
68813                    1   11706.42       11706.42      1086.93   
68814                    1    4666.78        4666.78       571.33   
68815                    0   36953.94       36953.94      3661.36   
68816                    1   15395.66       15395.66      1041.03   

       total_pymnt_inv  total_rec_prncp  total_rec_int  total_rec_late_fee  \
0               355.29           295.13          60.16                 0.0   
1               873.53           706.87         166.66                 0.0   
2               485.44           363.21         122.23                 0.0   
3               335.33           280.66          54.67                 0.0   
4               484.36           385.27          99.09                 0.0   
...                ...              ...            ...                 ...   
68812          1019.42           673.14         346.28                 0.0   
68813          1086.93           293.58         793.35 

In [11]:
y = y.to_frame()

In [12]:
# Check the balance of our target values
y['loan_status'].value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [13]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y )

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [14]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 51352, 0: 51352})

In [15]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

model_sample = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model_sample.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model_sample.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6594379378834943

In [17]:
# Display the confusion matrix
# YOUR CODE HERE
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   60,    27],
       [ 6347, 10771]], dtype=int64)

In [18]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.69      0.63      0.02      0.66      0.44        87
          1       1.00      0.63      0.69      0.77      0.66      0.43     17118

avg / total       0.99      0.63      0.69      0.77      0.66      0.43     17205



### SMOTE Oversampling

In [19]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, ratio=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled_smote)

Counter({1: 51352, 0: 51352})

In [20]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
# Calculated the balanced accuracy score
# YOUR CODE HEREy_pred = model.predict(X_test)
y_pred_smote = model_smote.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smote)

0.628576090503644

In [22]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_smote)

array([[   50,    37],
       [ 5436, 11682]], dtype=int64)

In [23]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.57      0.68      0.02      0.63      0.39        87
          1       1.00      0.68      0.57      0.81      0.63      0.40     17118

avg / total       0.99      0.68      0.58      0.81      0.63      0.40     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [24]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
# Fit the data using `ClusterCentroids` and check the count of each class
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled_under, y_resampled_under = cc.fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled_under)

Counter({0: 260, 1: 260})

In [25]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
# Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression

model_under = LogisticRegression(solver='lbfgs', random_state=1)
model_under.fit(X_resampled_under, y_resampled_under)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_under = model_under.predict(X_test)
balanced_accuracy_score(y_test, y_pred_under)

0.5207877571904549

In [27]:
# Display the confusion matrix
# YOUR CODE HERE
# Display the confusion matrix
confusion_matrix(y_test, y_pred_under)

array([[  51,   36],
       [9323, 7795]], dtype=int64)

In [28]:
# Print the imbalanced classification report
# YOUR CODE HERE
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_under))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.46      0.01      0.52      0.27        87
          1       1.00      0.46      0.59      0.62      0.52      0.26     17118

avg / total       0.99      0.46      0.59      0.62      0.52      0.26     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [29]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled_smoteenn, y_resampled_smoteen = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled_smoteen)

Counter({0: 51344, 1: 46388})

In [31]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model_smoteenn.fit(X_resampled_smoteenn, y_resampled_smoteen)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_smoteenn = model_smoteenn.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smoteenn)

0.6629077679877201

In [34]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_smoteenn)

array([[  65,   22],
       [7212, 9906]], dtype=int64)

In [35]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_smoteenn))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.75      0.58      0.02      0.66      0.44        87
          1       1.00      0.58      0.75      0.73      0.66      0.43     17118

avg / total       0.99      0.58      0.75      0.73      0.66      0.43     17205



#### Q: Which model had the best balanced accuracy score?

A: The combination sampling had the best accurary score.  
    
#### Q: Which model had the best recall score?

A: SMOTE Oversampling had the best recall score.
    
#### Q: Which model had the best geometric mean score?

A: Oversampling and Combination sampling had the same geo score higher than the rest.